In [30]:
import urllib
import json
from pprint import pprint
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [31]:
# Get the dataset metadata by passing package_id to the package_search endpoint
# For example, to retrieve the metadata for this dataset:

url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/package_show"
params = { "id": "8c233bc2-1879-44ff-a0e4-9b69a9032c54"}
response = urllib.request.urlopen(url, data=bytes(json.dumps(params), encoding="utf-8"))
package = json.loads(response.read())
pprint(package)

{'help': 'https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/help_show?name=package_show',
 'result': {'creator_user_id': '150d5301-86ec-44a3-a070-50f2cea839c9',
            'dataset_category': 'Document',
            'excerpt': 'This dataset contains non-identifiable information '
                       'relating to each parking ticket issued for each '
                       'calendar year since 2008.',
            'formats': 'XLS,ZIP',
            'groups': [],
            'id': '8c233bc2-1879-44ff-a0e4-9b69a9032c54',
            'information_url': 'http://www.toronto.ca/pay-toronto-tickets/parkingtickets/index.htm',
            'is_retired': False,
            'isopen': True,
            'last_refreshed': '2019-07-23T17:51:57.153365',
            'license_id': 'other-open',
            'license_title': 'Other (Open)',
            'metadata_created': '2019-07-23T17:50:14.331462',
            'metadata_modified': '2019-11-06T15:23:09.564050',
            'name': 'parking-ti

In [32]:
filedata_2018_url = package["result"]["resources"][10]["url"]
filedata_2018_url

'https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/8c233bc2-1879-44ff-a0e4-9b69a9032c54/resource/e0df3b42-8515-492b-ba85-a55c4cfd035f/download/parking-tickets-2018.zip'

In [33]:
# how to download the file from the above url
import requests
data_2018 = requests.get(filedata_2018_url)

In [34]:
#store the downloaded file in the parking2018.zip 
open('Resourses/parking2018.zip', 'wb').write(data_2018.content)

24403357

In [35]:
# extract all the files from the zip folder to current working directory
from zipfile import ZipFile

In [36]:
with ZipFile("Resourses/parking2018.zip","r") as zfile:
    zfile.extractall("Resourses")

In [37]:
data1 = pd.read_csv("Resourses/Parking_Tags_Data_2018_1.csv")
data2 = pd.read_csv("Resourses/Parking_Tags_Data_2018_2.csv")
data3 = pd.read_csv("Resourses/Parking_Tags_Data_2018_3.csv")

In [38]:
complete_data = pd.concat([data1,data2,data3])
complete_data.head()


,tag_number_masked,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,location3,location4,province
0,***92517,20180101,16,PARK-WITHIN 9M INTERSECT ROAD,50,0.0,S/S,PRYOR AVE,E/O,CLOVERDALE RD,ON
1,***71708,20180101,29,PARK PROHIBITED TIME NO PERMIT,30,2.0,NR,266 DOVERCOURT RD,NaN,NaN,ON
2,***92311,20180101,29,PARK PROHIBITED TIME NO PERMIT,30,2.0,NR,15 FAIRBANK AVE,NaN,NaN,ON
3,***92312,20180101,29,PARK PROHIBITED TIME NO PERMIT,30,2.0,NR,15 FAIRBANK AVE,NaN,NaN,ON
4,***71709,20180101,29,PARK PROHIBITED TIME NO PERMIT,30,3.0,NR,266 DOVERCOURT RD,NaN,NaN,ON


In [39]:
len(complete_data)

2044288

In [40]:
complete_data.dtypes

tag_number_masked          object
date_of_infraction          int64
infraction_code             int64
infraction_description     object
set_fine_amount             int64
time_of_infraction        float64
location1                  object
location2                  object
location3                  object
location4                  object
province                   object
dtype: object

In [41]:
wData11 = pd.read_csv("Resourses\en_climate_daily_ON_TorontoCityCenter_2016.csv")
wData12 = pd.read_csv("Resourses\en_climate_daily_ON_TorontoCityCenter_2017.csv")
wData13 = pd.read_csv("Resourses\en_climate_daily_ON_TorontoCityCenter_2018.csv")
wData21 = pd.read_csv("Resourses\en_climate_daily_ON_TorontoCity_2018.csv")
wData22 = pd.read_csv("Resourses\en_climate_daily_ON_TorontoCity_2018.csv")
wData23 = pd.read_csv("Resourses\en_climate_daily_ON_TorontoCity_2018.csv")

In [42]:
wDataTCC = pd.concat([wData11, wData12, wData13])
wDataTC = pd.concat([wData21, wData22, wData23])
weatherData = pd.merge(wDataTCC, wDataTC, on =["Date/Time", "Date/Time"] )
len(weatherData)

1095

In [43]:
combine_weatherData = pd.DataFrame(weatherData[["Date/Time", "Total Rain (mm)_x", "Total Snow (cm)_x", "Mean Temp (°C)_y" ]])
combine_weatherData.rename(columns={"Date/Time" : "Date", "Total Rain (mm)_x" :"TotalRain(mm)", "Total Snow (cm)_x": "TotalSnow(cm)", "Mean Temp (°C)_y" : "Mean Temp_in_Celsius"}, inplace = True)

combine_weatherData.head()

,Date,TotalRain(mm),TotalSnow(cm),Mean Temp_in_Celsius
0,2018-01-01,NaN,NaN,-13.3
1,2018-01-01,NaN,NaN,-13.3
2,2018-01-01,NaN,NaN,-13.3
3,2018-01-02,NaN,NaN,-9.8
4,2018-01-02,NaN,NaN,-9.8


In [45]:
final_df = pd.merge(complete_data, combine_weatherData, how='left', on=["date_of_infraction", "Date"])

KeyError: 'date_of_infraction'